# Executors and Tasks

This notebook has a brief explanation of how we are going to create generic **Classes** capable of handling important Data Lake jobs. *E.g.*, transferring data from different storages, creating tables on **AWS Athena** and **AWS Redshift**, managing **Glue Jobs**, *etc*.

## Staging Data with a optional transformation

The class we are creating below is capable of taking data from a parent directory and moving it to a dump directory. Furthermore, given a python script, it will run it with its respective arguments.

In [1]:
class StagingExecutor:
    
    def __init__(self, parent_directory: str, dump_directory: str, archive_or_delete: str = "archive", py_exec_path: str = None, py_exec_args: dict = None) -> None:
        self.parent = parent_directory
        self.dump = dump_directory
        self.archive_or_delete = archive_or_delete
        self.py_exec_path = py_exec_path
        self.py_exec_args = py_exec_args

    def transfer(self) -> None:
        
        if not self.py_exec_path:
            print('No transformation required. Moving file using only parent and dump')
        
        elif self.py_exec_path:
            import sys
            sys.path.insert(1, self.py_exec_path)
            import py_exec
            
            py_exec.test()
            
            if not self.py_exec_args:
                print('Python Executor does not require arguments')
            
            elif self.py_exec_args:
                print(f'Python Executor is running with the following parameters:\n{self.py_exec_args}')
                py_exec.main(self.parent, self.dump, **self.py_exec_args)
                
    def post_staging(self) -> None:
                
        if self.archive_or_delete == 'archive':
            print('File from landing will be moved to archive folder')
        
        elif self.archive_or_delete == 'delete':
            print('File will be deleted from landing')

In [2]:
import json

import sys
sys.path.insert(1, r'..')

import executors

f = '../routines/test_routine/routine_config.json'

with open(f, 'r') as j:
    routine_config = json.loads(j.read())

# Orchestrator

Data orchestration is a relatively new concept to describe the set of technologies that abstracts data access across storage systems, virtualizes all the data, and presents the data via standardized APIs with a global namespace to data-driven applications. There is a clear need for data orchestration because of the increasing complexity of the data ecosystem due to new frameworks, cloud adoption/migration, as well as the rise of data-driven applications. [[Data Orchestrator]](https://dzone.com/articles/data-orchestration-its-open-source-but-what-is-it)

The Orchestrator is a class that will follow the routine_config.json, where one will declare which executors and their respective tasks to run. The model for our routine_config is:

```
{
    "routine_name": <ROUTINE_NAME>,
    "executors": {
        <EXECUTOR_CLASS:  {
            "params":<__init__ PARAMETERS>,
            "tasks": <LIST_SELECTED_TASKS_FROM_EXECUTOR>
        }
    }
}   
```

In [3]:
class Orchestrator:
    
    def __init__(self, routine_config: dict) -> None:
        self.routine_config = routine_config
    
    def run_executors(self):
        for executor in routine_config['executors']:
            self.executor_name = executor
            klass = globals()[self.executor_name]
            self.executor = klass(**routine_config['executors'][self.executor_name]['params'])
            self.run_tasks()
            
    def run_tasks(self):
        for task in routine_config['executors'][self.executor_name]['tasks']:
            getattr(self.executor, task)()

In [4]:
orchestrator = Orchestrator(routine_config)

In [6]:
orchestrator.run_executors()

This is running inside other module, and being executed because of a call.
For compliance: Hello World (◕‿◕✿)
Test ok
Python Executor is running with the following parameters:
{'mapper': {'column1': 'name', 'column2': 'age', 'column3': 'job'}}
File will be deleted from landing
